In [1]:
# Estudo paradas tratadas
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import box, Point, Polygon, MultiPolygon
from shapely.wkt import loads
import folium as fl
import pandas as pd

In [78]:
dt = pd.read_csv("../../../databases/luiz_dataset/paradas_linestrings_ida.csv", converters={'parada': pd.eval})
linhas = pd.read_csv('../../../databases/linhas/Linhas.csv')

In [79]:
# remove as colunas
dt.drop(columns='Unnamed: 0',axis=0)

,linha,parada
0,0.018,"[[-15.747487243955707, -47.89600866323914, 351..."
1,0.946,"[[-15.80778852625573, -47.96642705712294, 2256..."
2,0.373,"[[-15.866555379423424, -47.96504713913905, 605..."
3,503.300,"[[-15.638937404577504, -47.82260762728512, 491..."
4,531.100,"[[-15.638937404577504, -47.82260762728512, 491..."
...,...,...
482,0.844,"[[-15.884075085352835, -48.129561044093, 4228]..."
483,0.172,"[[-15.866555379423424, -47.96504713913905, 605..."
484,805.600,"[[-15.867119301987762, -48.02312295895697, 534..."
485,506.200,"[[-15.638004360182045, -47.821280315734576, 48..."


In [80]:
# Faz o teste para ver se tem uma linha no seu dataframe
if '0.813' in [str(dado) for dado in list(dt.linha.unique())]:
    print('Tem')
else:
    print('Não tem')

Tem


In [84]:
# Fazendo o plot do 813
dt_813 = dt[dt['linha']==0.205]

In [76]:
dt_813 = dt_813.drop(columns='Unnamed: 0',axis=0)

In [91]:
# Salvando a rota em um array de tuplas
for ponto in dt_813.parada:
    m = fl.Map(location=[-15.76050, -47.78134], zoom_start=10)
    for lat, long, id_name in ponto:
        fl.Circle([lat, long], fill_color='#FF0000', radius=15).add_to(m)
    break
m    

In [ ]:
  # Fazer uma média entre a distÂncia entre as paradas, fazer um trashold para para limitar as paradas que são pegas 
  # menores que a média.
  
  
coord_list = [(x,y) for x,y in zip(gdf['geometry'].x , gdf['geometry'].y)]